In [1]:
# !pip install torch sentencepiece accelerate bitsandbytes git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/peft.git

In [1]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    llm_int8_enable_fp32_cpu_offload=True, 
    load_in_8bit=True,
)

MODEL_NAME = "IlyaGusev/llama_7b_ru_turbo_alpaca_lora"

config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    device_map="auto",
    quantization_config=quantization_config,
)
model = PeftModel.from_pretrained(model, MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/home/dimweb/Desktop/deeppavlov/ai_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.8/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/dimweb/Desktop/deeppavlov/ai_env/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
Loading checkpoint shards: 100%|██████████| 33/33 [00:06<00:00,  4.72it/s]


In [2]:
model.device

device(type='cuda')

In [4]:
inputs = [
    "Вопрос: Почему трава зеленая?\n\nВыход:",
    "Задание: Сочини длинный рассказ, обязательно упоминая следующие объекты.\nВход: Таня, мяч\nВыход:",
    "Могут ли в природе встретиться в одном месте белый медведь и пингвин? Если нет, то почему?\n\n",
    "Задание: Заполни пропуски в предложении.\nВход: Я пытался ____ от маньяка, но он меня настиг\nВыход:",
    "Как приготовить лазанью?\n\n",
    "Реши уравнение 4x + 5 = 21"
]

In [5]:
for inp in inputs:
    data = tokenizer([inp], return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items() if k in ("input_ids", "attention_mask")}
    output_ids = model.generate(
        **data,
        num_beams=3,
        max_length=256,
        do_sample=True,
        top_p=0.95,
        top_k=40,
        temperature=1.0,
        repetition_penalty=1.2,
        no_repeat_ngram_size=4,
        
    )[0]
    print(tokenizer.decode(output_ids, skip_special_tokens=True))
    print()
    print("==============================")
    print()

/home/dimweb/Desktop/deeppavlov/ai_env/lib/python3.11/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'device'

: 